In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26646153
paper_name = 'choy_basrai_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [26]:
original_data_list = []
for p in np.arange(14)+1:
    dt = pd.read_excel('raw_data/supp_g3.115.022244_TableS2.xlsx', sheet_name='S1 P' + str(p))
    dt.columns = [c.lower() for c in dt.columns]
    original_data_list.append(dt)

In [27]:
for p in np.arange(14)+1:
    dt = pd.read_excel('raw_data/supp_g3.115.022244_TableS3.xlsx', sheet_name='S2 P' + str(p))
    dt.columns = [c.lower() for c in dt.columns]
    original_data_list.append(dt)

In [28]:
len(original_data_list)

28

In [29]:
original_data = pd.concat(original_data_list, axis=0, ignore_index=True)

In [30]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 8782 x 6


In [31]:
original_data.head()

,array orf,array name,array annotation,normalized colony size (experiment),normalized colony size (control),ratio
0,YBR200W,BEM1,NaN,0.04163,0.61013,0.068231
1,YAR015W,ADE1,NaN,0.02722,0.22616,0.120357
2,YBR189W,RPS9B,NaN,0.08463,0.47900,0.176681
3,YBL027W,RPL19B,NaN,0.18382,0.71986,0.255355
4,YBL095W,YBL095W,NaN,0.25758,0.72932,0.353178


In [32]:
original_data['orf'] = original_data['array orf'].astype(str)

In [33]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [34]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [35]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            array orf array name  array annotation  \
index_input                                          
78                 33         33               NaN   
186                17         17               NaN   
223                 1          1               NaN   
298                65         65               NaN   
301                49         49               NaN   
...               ...        ...               ...   
8491                1          1               NaN   
8572               33         33               NaN   
8623               65         65               NaN   
8643               49         49               NaN   
8690               17         17               NaN   

             normalized colony size (experiment)  \
index_input                                        
78                                       0.99728   
186                                      1.05555   
223                                      1.14422   
298                                  

In [37]:
original_data = original_data.loc[t,:]

In [38]:
original_data['data'] = original_data['ratio']

In [39]:
original_data.set_index('orf', inplace=True)

In [40]:
original_data = original_data[['data']].copy()

In [41]:
original_data = original_data.groupby(original_data.index).mean()

In [42]:
original_data.shape

(4268, 1)

# Prepare the final dataset

In [43]:
data = original_data.copy()

In [44]:
dataset_ids = [768]
datasets = datasets.reindex(index=dataset_ids)

In [45]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [46]:
data.head()

dataset_id,768
data_type,value
orf,
YAL002W,0.736898
YAL004W,0.986907
YAL005C,0.916630
YAL007C,0.973737
YAL008W,0.887781


## Subset to the genes currently in SGD

In [47]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [48]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,768
,data_type,value
gene_id,orf,
2,YAL002W,0.736898
1863,YAL004W,0.986907
4,YAL005C,0.916630
5,YAL007C,0.973737
6,YAL008W,0.887781


# Normalize

In [49]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [50]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [51]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            768          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.736898 -1.242973
1863    YAL004W  0.986907 -0.160917
4       YAL005C  0.916630 -0.465079
5       YAL007C  0.973737 -0.217918
6       YAL008W  0.887781 -0.589941

# Print out

In [52]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [53]:
from IO.save_data_to_db3 import *

In [54]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 26646153...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.14s/it]

Updating the data_modified_on field...
